In [ ]:
### 241112 수정 

In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
import os

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

### chain이란?
- 한 task의 출력이 다음 task의 input이 됨 
- chain 종류 3가지. (어떤 chain을 구성할지에 대한)
  - utility
  - generic 
  - combine document
 

#### ex. prompt에 수식을 계산해달라고 하면? 
- 계산을 실제로 하는게 아니라 패턴을 학습하는 것임.
- 여러 계산 수식을 하는 원리가 학습되어 있음. 
- lstm으로 계산을 예측함. 
- 플러그인>수식을 생성>플러그인 사용>답>llm 답변 생성

### 1. utility chain

In [3]:
# LLMMathChain: 수학 계산을 위한 Utility Chain

from langchain_openai import ChatOpenAI
from langchain.chains import LLMMathChain
openai = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
    temperature = 0, 
    api_key = OPENAI_API_KEY
)

In [4]:
llm_math = LLMMathChain.from_llm(openai, verbose = True) # verbose 상세 로깅 활성화 
llm_math.invoke("What is 13 raised to the .3432 power?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13 ** 0.3432
```
...numexpr.evaluate("13 ** 0.3432")...

Answer: 2.4116004626599237
> Finished chain.


{'question': 'What is 13 raised to the .3432 power?',
 'answer': 'Answer: 2.4116004626599237'}

In [5]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



### 2. generic chain

In [ ]:
import re

In [7]:
# 정규식을 이용해 주어진 텍스트에서 반복되는 줄바꿈을 하나로 변환하고, 탭을 스페이스로 변환하는 함수를 작성

def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text) # 여러 개의 줄바꿈(\r\n, \r, \n)이 2개 이상 연속되면, 이를 하나의 \n으로 대체
    text = re.sub(r'[ \t]+', ' ', text) # 여러 개의 공백 또는 탭 문자가 연속되면, 이를 하나의 공백으로 대체
    return {"output_text": text}

In [8]:
from langchain.chains import TransformChain

# TransformChain: langchain 라이브러리에서 제공하는 변환 체인 객체

clean_extra_spaces_chain = TransformChain(input_variables=["text"],
                                          output_variables=["output_text"],
                                          transform=transform_func
                                         )

In [10]:
clean_extra_spaces_chain.run('A random text with some irregular spacing.\n\n\n Another one here as well.')

C:\Users\82103\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'A random text with some irregular spacing.\n Another one here as well.'

In [1]:
# chain.run 지원 x invoke 사용 
clean_extra_spaces_chain.invoke('A random text with some irregular spacing.\n\n\n Another one here as well.')

NameError: name 'clean_extra_spaces_chain' is not defined

### 3. sequential chain

- PromptTemplate과 LLMChain을 이용해 원하는 스타일로 텍스트를 변환하는 프롬프트 작성
- SequentialChain으로 TransformChain과 LLMChain을 연결

In [12]:
from langchain import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

# template: 텍스트를 특정 스타일로 바꿔서 표현하도록 지시

template = """Paraphrase this text:
{output_text}
In the style of a {style}.
Paraphrase: """

C:\Users\82103\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [14]:
prompt = PromptTemplate(input_variables = ["style", "output_text"], template = template) 

# LLMChain: LLM을 사용하여 주어진 프롬프트를 실행하는 체인

style_paraphrase_chain = LLMChain(llm = openai, prompt = prompt,
                                  output_key = 'final_output'
                                 ) 
# final_output: 체인의 출력

In [15]:
# 여러 chain 순차적 실행 
sequential_chain = SequentialChain(chains = [clean_extra_spaces_chain, # 공백 제거 체인
                                             style_paraphrase_chain], # 특정 스타일로 변환하는 체인
                                             input_variables = ['text', 'style'], # text, style이 입력 
                                             output_variables = ['final_output'] # final_ouptut에 최종 결과 담음
                                  )

In [13]:
input_text = """
Chains allow us to combine multiple \n\n\n
components together to create a single, coherent application. \n\n
For example, we can create a chain that takes user input, \t format it with a
PromptTemplate, \n\n
and then passes the formatted response to an LLM. We can build more complex chains by
combining \t multiple chains together, or by \n\n\n\n
combining chains with other components.
"""

sequential_chain.invoke({'text': input_text, 'style': 'a 90s rapper'}) # 체인을 순서대로 실행 

{'text': '\nChains allow us to combine multiple \n\n\n\ncomponents together to create a single, coherent application. \n\n\nFor example, we can create a chain that takes user input, \t format it with a\nPromptTemplate, \n\n\nand then passes the formatted response to an LLM. We can build more complex chains by\ncombining \t multiple chains together, or by \n\n\n\n\ncombining chains with other components.\n',
 'style': 'a 90s rapper',
 'final_output': "Chains be like the glue that sticks all the pieces together, makin' one dope app. Like, we can make a chain that grabs what the user types, dresses it up with a PromptTemplate, and then hands it off to an LLM. We can get even fancier by mixin' and matchin' different chains, or throwin' in other components for extra flavor."}

In [20]:
result = sequential_chain.invoke({'text': input_text, 'style': 'a 90s rapper'})
print(result['final_output'])

Chains be like mixin' different parts to make one dope app. Like, we can put together a chain that takes what the user says, jazz it up with a PromptTemplate, and then send it to an LLM. We can get even fancier by mixin' a bunch of chains or throwin' in other components.
